In [16]:
# load dataset 
import pandas as pd 
from sklearn.model_selection import train_test_split
pathLyrics = 'lyrics-data.csv'
pathArtists = 'artists-data.csv'


dataframe = pd.read_csv(pathLyrics)
dataframeArtists = pd.read_csv(pathArtists)

In [17]:
# filter Idiom for olny english lyrics
dataframe = dataframe[dataframe['Idiom'] == 'ENGLISH']

In [18]:
# normalize lycris to lowercase  
dataframe['Lyric'] = dataframe['Lyric'].apply(lambda lyric: lyric.lower())

# Zwei csv werden gemerged
mergeData=pd.merge(dataframe, dataframeArtists, how='inner', left_on='ALink', right_on='Link')

#Filtern der Daten. Nur Lyric und Genre für das Dataset notwendig
rawData= mergeData[['Lyric', 'Genre']]

# Speichern der csv zum Anschauen
# dataset.to_csv('dataset.csv') 

In [19]:
# Es werden die Spalten gefiltert, die Noten enthalten
rawData = rawData[~rawData['Lyric'].str.contains('----')] 

#Filtert alle Zeichen bis auf Buchstaben und Zahlen heraus
rawData['Lyric']=rawData['Lyric'].str.replace('[^a-zA-Z0-9]', ' ')


In [33]:
train, test = train_test_split(rawData, test_size=0.2)



In [45]:
from collections import Counter 
word_raw = Counter(" ".join(train["Lyric"]).split()).most_common(10000)
word_index = []

for i in word_raw:
        word_index.append(i[0])
        


['i',
 'you',
 'the',
 'to',
 'and',
 'it',
 'a',
 'me',
 't',
 's',
 'my',
 'in',
 'that',
 'm',
 'on',
 'your',
 'we',
 'of',
 'all',
 'can',
 'be',
 'don',
 'is',
 'like',
 'for',
 'know',
 'so',
 'love',
 'up',
 'but',
 'with',
 'just',
 'what',
 'this',
 'oh',
 're',
 'no',
 'got',
 'do',
 'get',
 'when',
 'now',
 'll',
 'if',
 'go',
 'she',
 'out',
 'they',
 'let',
 'down',
 'yeah',
 'baby',
 'one',
 'see',
 'time',
 'there',
 'never',
 'want',
 'cause',
 'not',
 'was',
 'are',
 'make',
 'say',
 've',
 'come',
 'wanna',
 'back',
 'take',
 'way',
 'have',
 'at',
 'from',
 'girl',
 'he',
 'will',
 'ain',
 'how',
 'her',
 'gonna',
 'right',
 'feel',
 'here',
 'need',
 'life',
 'tell',
 'as',
 'away',
 'been',
 'could',
 'more',
 'give',
 'night',
 'who',
 'where',
 'heart',
 'about',
 'man',
 'keep',
 'why',
 'think',
 'world',
 'day',
 'good',
 'too',
 'some',
 'then',
 'only',
 'ya',
 'by',
 'd',
 'or',
 'won',
 'chorus',
 'them',
 'around',
 'look',
 'little',
 'through',
 'over'

In [ ]:
#word_index = imdb.get_word_index()
#reverse_word_index = dict([(value, key) for (key, value) in word_raw.items()])

#decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])
#print("Example review: " + decoded_review)

for i in train["Lyric"]:
    for word in word_index:
        i.replace(word, "TOBITOBI")
    print(i)